<a href="https://colab.research.google.com/github/yhatpub/yhatpub/blob/notebook/notebooks/fastai/lesson2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fastai Lesson 2 on YHat.pub

This notebook picks up from [Fastai Fastbook 02_production](https://github.com/fastai/fastbook/blob/master/02_production.ipynb) publishing your bear classifier to [YHat.pub](https://yhat.pub)

In [1]:
!pip install -q --upgrade --no-cache-dir torch torchvision torchaudio
!pip install -q --upgrade --no-cache-dir fastai

     |████████████████████████████████| 831.4 MB 1.4 MB/s 
     |████████████████████████████████| 22.1 MB 1.4 MB/s 
     |████████████████████████████████| 1.9 MB 45.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.9.1 which is incompatible.
     |████████████████████████████████| 186 kB 5.2 MB/s 
     |████████████████████████████████| 56 kB 45.6 MB/s 


In [2]:
from fastai.vision.all import *

In [3]:
!pip install -Uqq --no-cache-dir git+https://github.com/yhatpub/yhat_params.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 131 kB 5.0 MB/s 
     |████████████████████████████████| 62 kB 15.2 MB/s 
     |████████████████████████████████| 8.0 MB 48.1 MB/s 
     |████████████████████████████████| 79 kB 34.5 MB/s 
     |████████████████████████████████| 138 kB 44.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.9.1 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
from yhat_params.yhat_tools import inference_test, FieldType, inference_predict

In [5]:
!wget -q --no-check-certificate 'https://docs.google.com/uc?export=download&id=1tRoH4Yxg8izIlX7bCrRTwS13wHb0kiJT' -r -A 'uc*' -e robots=off -nd
!mkdir -p model
!mv $(ls -S uc* | head -1) ./model/export.pkl

In [ ]:
!ls -l ./model/export.pkl

In [ ]:
learn_inf = load_learner('./model/export.pkl')

In [ ]:
input = {"image": FieldType.PIL}
output = {"text": FieldType.Text}

@inference_predict(input=input, output=output)
def predict(params):
    img = PILImage.create(np.array(params["image"].convert("RGB")))
    result = learn_inf.predict(img)
    return {"text": str(result[0])}

In [ ]:
from yhat_params.yhat_tools import in_colab

if in_colab():
    import urllib.request
    from PIL import Image
    urllib.request.urlretrieve("https://upload.wikimedia.org/wikipedia/commons/thumb/a/a9/GrizzlyBearJeanBeaufort.jpg/220px-GrizzlyBearJeanBeaufort.jpg", "input_image.jpg")
    img = Image.open("input_image.jpg")
    inference_test(predict_func=predict, params={'image': img})